## 製品価格予測

（T4 で実行可能）

### 第7週 一日目

LoRA と QLoRAの紹介

## リマインダー：Colabを使用するための2つの重要なプロチップ：

**Pro-tip 1:**

すべてのColabの上部には、いくつかのpipインストールがあります。これを実行すると、pipからエラーを受信する場合があります。

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

これらのpip互換性エラーは安全に無視できます。そして、バージョン番号を変更することでそれらを修正しようとするのは魅力的ですが、実際に実際の問題を導入します！

**Pro-tip 2:**

Colabを実行している最中に、次のようなエラーが発生する可能性があります。

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

これは非常に誤解を招くエラーメッセージです。パッケージのバージョンを変更しないでください。

これは、GoogleがColabランタイムを切り替えたために実際に起こります。おそらくGoogle Colabが忙しすぎたためです。解決策は次のとおりです。

1. カーネルメニュー >> ランタイムを切断して削除
2. 新鮮なメニューからColabをリロードし、メニュー >> すべての出力をクリア
3. 右上のボタンを使用して新しいT4に接続
4. 右上のメニューから「リソースを表示」を選択して、GPUがあることを確認
5. Colab内のセルを上から下へ、pipのインストールから始めて再実行

そして、すべてがうまくいくはずです - そうでなければ、私に尋ねてください！

In [ ]:
# pip install

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0
!pip install -q datasets requests peft

In [ ]:
# import

import os
import re
import math
from datetime import datetime
from tqdm import tqdm

from google.colab import userdata
from huggingface_hub import login

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel

In [ ]:
# 定数

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"ed-donner/pricer-2024-09-13_13.04.39"

# QLoRAファインチューニングのためのハイパーパラメタ

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

### Hugging Faceにログインします

まだHugging Faceアカウントをお持ちでない場合は、https://huggingface.co にアクセスしてサインアップしてトークンを作成します。

次に、左のキーアイコンをクリックして、このノートブックのシークレットを選択し、トークンとして値を持つ「HF_TOKEN」と呼ばれる新しい秘密を追加します。

In [ ]:
# Hugging Faceにログイン

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
base_model = None

## さまざまな量子化を試します

In [ ]:
# 量子化なしでトークナイザとベースモデルをロード
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

上記を実行して得られる出力
```
config.json: 100% ■■■ 826/826 [00:00<00:00, 77.7kB/s]
model.safetensors.index.json: 100% ■■■ 23.9k/23.9k [00:00<00:00, 882kB/s]
Downloading shards: 100% ■■■  4/4 [20:02<00:00, 265.40s/it]
model-00001-of-00004.safetensors: 100% ■■■ 4.98G/4.98G [04:39<00:00, 20.0MB/s]
model-00002-of-00004.safetensors: 100% ■■■ 5.00G/5.00G [06:55<00:00, 29.9MB/s]
model-00003-of-00004.safetensors: 100% ■■■ 4.92G/4.92G [07:28<00:00, 21.4MB/s]
model-00004-of-00004.safetensors: 100% ■■■ 1.17G/1.17G [00:58<00:00, 108MB/s]
Loading checkpoint shards: 100% ■■■ 4/4 [00:55<00:00, 27.63s/it]
generation_config.json: 100% ■■■ 185/185 [00:00<00:00, 5.54kB/s]
WARNING:accelerate.big_modeling:Some parameters are on the meta device because they were offloaded to the cpu and disk.
```

In [ ]:
# T4 の GPU RAM は 15GB 、当該モデルが 32.1 GB を占拠 ... T4 ではできないので、
# 一部（32.1 - 11.0 = 21.0 GB）のモデル重みを一時的にディスクやCPUに退避させている。
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

上記を実行して得られる出力
```
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)
```

## セッションを再開してください！

次のモデルをロードする前に、最後のモデルのキャッシュをクリアするには、ランタイム>>セッションを再起動し、初期セル（インストールとインポート、およびHugging Face Hubログイン）を再度実行する必要があります。

これはGPUをクリーンアウトするためです。

In [ ]:
# 8ビットを使用してトークナイザとベースモデルをロード

quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

上記を実行して得られる出力（キャッシュを使いダウンロードはしない）
```
Loading checkpoint shards: 100% ■■■ 4/4 [01:32<00:00, 19.80s/it]
```

In [ ]:
# T4 の GPU RAM は 15GB 、当該モデルが 9.1 GB を占拠
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

上記を実行して得られる出力
```
WinMerge で確認すると Linear → Linear8bitLt に変わっていることが確認できる。
```

## セッションを再開してください！

次のモデルをロードする前に、最後のモデルのキャッシュをクリアするには、ランタイム>>セッションを再起動し、初期セル（インストールとインポート、およびHugging Face Hubログイン）を再度実行する必要があります。

これはGPUをクリーンアウトするためです。

In [ ]:
# 4ビットを使用してトークナイザとベースモデルをロード

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

上記を実行して得られる出力（キャッシュを使いダウンロードはしない）
```
Loading checkpoint shards: 100% ■■■ 4/4 [01:40<00:00, 22.34s/it]
```

In [ ]:
# T4 の GPU RAM は 15GB 、当該モデルが 5.59 GB を占拠
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
base_model

上記を実行して得られる出力
```
WinMerge で確認すると Linear → Linear8bitLt → Linear4bit 変わっていることが確認できる。
```

※ ココでは、セッションを再開しない。

In [ ]:
# ファインチューニング後のトークナイザとベースモデルをロード
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

上記を実行して得られる出力（キャッシュを使いダウンロードはしない）
```
adapter_config.json: 100% ■■■ 681/681 [00:00<00:00, 39.0kB/s]
adapter_model.safetensors: 100% ■■■ 109M/109M [00:00<00:00, 221MB/s]
```

In [ ]:
# T4 の GPU RAM は 15GB 、当該モデルが 5.70 GB を占拠
# 従って,QLoRAのパラメタは、5.70 - 5.59 = 0.11GB = 110MB と思われる。
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

上記を実行して得られる出力、lora_q、lora_k、lora_v、lora_o に QLoRAが適用されている。
```
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (v_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
            (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          )
        )
        (norm): LlamaRMSNorm((4096,), eps=1e-05)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
    )
  )
)
```

以下のセルで計算してみると、凡そ合っている事が解る。
```
Total number of params: 27,262,976 and size 109.1MB
```

In [ ]:
# 各ターゲットモジュールには、LORA_AとLORA_Bと呼ばれる2つのLORAアダプターマトリックスがあります
# これらは、alpha * lora_a * lora_bを追加することでウェイトを適応できるように設計されています

# 次のサイズを使用して、重みの数を数えてみる。

# 上記のマトリックス寸法を参照してください
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# 各レイヤーが登場します
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# 32のレイヤーがあります
params = lora_layer * 32

# したがって、MBの合計サイズはです
size = (params * 4) / 1_000_000

print(f"Total number of params: {params:,} and size {size:,.1f}MB")